# Sports Book: Over/Under Number of Generations By Map

(well, by cup, and by map...)

This notebook uses the Game Outcomes Table (SQL) to compute the number of generations by map. The 50th quantile of the empirical CDF (the median of the number of generations) gives the over/under value.

This data is used to create a new table (with a corresponding data class given below) that stores expected number of generations by map and by cup.

In [7]:
import os, sys, subprocess, json, time
import statistics
from pprint import pprint

In [8]:
from sqlalchemy import Column, Integer, String, Boolean, Table
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import func, select, column

In [9]:
# SQL copypasta

class Base(DeclarativeBase):
    pass
    
class GameResult(Base):
    __tablename__ = "sportsbook_game_result"
    gameid = Column("gameid", String, primary_key=True)
    cup = Column("cup", String)
    season = Column("season", Integer)
    day = Column("day", Integer)
    mapName = Column("mapName", String)
    winnerAbbr = Column("winnerAbbr", String)
    winnerPoints = Column("winnerPoints", Integer)
    loserAbbr = Column("loserAbbr", String)
    loserPoints = Column("loserPoints", Integer)
    isPostseason = Column("isPostseason", Boolean)
    series = Column("series", String)
    generations = Column("generations", Integer)

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

fpath = os.path.join(os.environ['HOME'], 'tmp', 'sqlite', 'mydb.db')
engine = create_engine(f"sqlite:///{fpath}")
Session = sessionmaker(engine)
session = Session()

In [11]:
class MapGenerations(Base):
    __tablename__ = "sportsbook_stats_generations"
    id = Column("id", Integer, primary_key=True)
    cup = Column("cup", String)
    mapName = Column("mapName", String)
    # This is the median number of generations scored on this map
    generations = Column("generations", Integer)

In [12]:
# Drop all prior results and re-calculate
session.query(MapGenerations).delete()

0

In [13]:
if session.query(MapGenerations).count() == 0:
    print(f"Confirmed that table {MapGenerations.__tablename__} is empty")

Confirmed that table sportsbook_stats_generations is empty


In [14]:
def add_map_generations(this_cup, this_mapName):
    query = select(GameResult.generations).where(GameResult.cup==this_cup).where(GameResult.mapName==this_mapName)
    res = session.execute(query)
    values = [j[0] for j in res]
    med = int(statistics.median(values))

    # Stash median value in the Map Generations Table
    # #print(f"Inserting median value: {this_cup} - {this_mapName} - {med}")
    g = MapGenerations(
        cup = this_cup,
        mapName = this_mapName,
        generations = med
    )
    session.add(g)

In [15]:
# Get unique cup-mapName combinations, and iterate over each one
stmt = select(GameResult.cup, GameResult.mapName).distinct().order_by(GameResult.cup, GameResult.mapName)
for irow, row in enumerate(session.execute(stmt)):
    this_cup = row[0]
    this_mapName = row[1]
    # Pass to the function that determines median number of generations for this cup-map combo, and inserts it
    add_map_generations(this_cup, this_mapName)

In [16]:
if session.query(MapGenerations).count() > 0:
    print(f"Confirmed that table {MapGenerations.__tablename__} is no longer empty")

Confirmed that table sportsbook_stats_generations is no longer empty
